# find-about-pages
SanjayKAroraPhD@gmail.com <br>
December 2018

## Description
Most of the interesting company information is not on home pages but on about us pages.  This script finds those links from the home pages for further crawling. 

## Change log
v3 adds the python boilerplate  api for web page cleaning.

## TODO:
* Need to make better use of all pages in the site, e.g., to improve data quality and use additional paragraph data found on non-homepages 
* Standardize firm name cleaning code utility into a universally accessible module
* Train a more effective boilerplate model to capture meaningful text from firm web pages

In [85]:
# import data processing and other libraries
import csv
import sys
import requests
import os
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
import io
from IPython.display import display
import time
import numpy as np
from bs4 import BeautifulSoup
import string
from urlparse import urlparse
import requests

In [2]:
from boilerpipe.extract import Extractor

In [3]:
MONGODB_DB = "FirmDB_20181203"
MONGODB_COLLECTION = "pages_depth0"
CONNECTION_STRING = "mongodb://localhost"
username = "scrapy"
password = "eager"
authSource = "FirmDB"
authMechanism='SCRAM-SHA-1'

client = pymongo.MongoClient(CONNECTION_STRING, username=username, password=password, authSource=authSource, authMechanism=authMechanism)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

DATA_DIR = '/Users/sarora/dev/EAGER/data/orgs/depth0_boilerpipe/'

In [142]:
# gather unique firm_names from mongodb

def get_firm_aggregates ():
    query = [ { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    # test for a specific firm
    # query = [ {"$match":{"firm_name": 'Dymax Corporation'}}, { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    results = col.aggregate(query)

    mongo_dict = {}
    for result in results:
        key = (result['_id'])
        if key:
            mongo_dict[key[0]] = result['number']
    
    return mongo_dict

results_dict = get_firm_aggregates()
firm_names = results_dict.keys()
print (len(firm_names))
pp = pprint.PrettyPrinter()
pp.pprint(firm_names)

1152
[u'Little Kids',
 u'Daylight Solutions',
 u'Biocon Limited',
 u'Sony Corporation',
 u'Magna International Inc.',
 u'Supremex Inc.',
 u'Honeywell International Inc.',
 u'Aurrion',
 u'The Jackson Laboratory',
 u'Hewlett Packard Enterprise Development LP',
 u'CUMMINS FILTRATION IP',
 u'Renewable Power Conversion',
 u'Fuji Electric Co.',
 u'Canon Kabushiki Kaisha',
 u'Sola U.S.A. Inc.',
 u'Chromalox',
 u'Empire Technology Development LLC',
 u'Cabot Corporation',
 u'aTyr Pharma',
 u'Alcon Research',
 u'Wyatt Technology Corporation',
 u'Southwest Research Institute',
 u'Skidmore',
 u'Fairchild Semiconductor Corporation',
 u'Saint-Gobain Performance Plastics Corporation',
 u'Ideal Power Inc.',
 u'NuOrtho Surgical',
 u'Suncore Photovoltaics',
 u'L-3 Communications Cincinnati Electronics Corporation',
 u'Medical Diagnostic Laboratories',
 u'Solarmer Energy',
 u'Braun Intertec Geothermal',
 u'ProNAi Therapeutics',
 u'Amicus Therapeutics',
 u'Raytheon Company',
 u'Weatherford Canada Partners

 u'Auterra',
 u'Sumitomo Electric Device Innovations',
 u'NovaSolix',
 u'Pharmatrophix',
 u'Microchip Technology Incorporated',
 u'Sima Therapeutics',
 u'Genomic Health',
 u'Carestream Health',
 u'Valicor',
 u'Avtaec Limited',
 u'GTherm',
 u'Saint-Gobain Adfors Canada',
 u'Evernote Corporation',
 u'Cree',
 u'IDEX Health & Science LLC',
 u'Qiagen GmbH',
 u'Echogen Power Systems',
 u'Lion Copolymer Geismar',
 u'Nanoridge Materials',
 u'Rebellion Photonics',
 u'Big Belly Solar',
 u'Nokomis',
 u'G.D.O. Inc',
 u'BTU International',
 u'VINDICO NANOBIO TECHNOLOGY INC.',
 u'Swagelok Company',
 u'Magnolia Optical Technologies',
 u'PolyPlus Battery Company',
 u'Ford Global Technologies',
 u'Bayer Cropscience AG',
 u'EMD Technologies Inc.',
 u'Nederlandse Organisatie voor toegepast-natuurwetenschappelijk onderzoek TNO',
 u'Nexcom Technology',
 u'AT&T Corporation',
 u'Floadia Corporation',
 u'Entech Solar',
 u'Amgen Fremont Inc.',
 u'PortaFire',
 u'Ormat Technologies Inc.',
 u'JAC Products Inc.',


In [162]:
# find all links that have about, company, and corporate mentions 
# 1. pull down home page
# 2. get a list of all links in Beautiful Soup
# 3. keep only those that match a regex

firm_about_dict = {}

def fix_url(url, domain):
    if not(re.search(domain, url)): 
        url = re.sub ('///?', '/', url)
        fixed = domain + '/' + url
        return fixed
    else:
        return url

def get_domain (url):
    o = urlparse(url)
    domain = o.scheme + '://' + o.netloc
    return domain

def get_link_path (url):
    o = urlparse(url)
    path = o.path + o.params + o.query + o.fragment
    return path

def get_about_links_from_home_pages (): 
    for firm_name in firm_names: 
        regex = '^' + re.escape(firm_name) + '$'
        results = col.find( {"firm_name": re.compile(regex, re.IGNORECASE)}).sort("depth",pymongo.ASCENDING) 

        result = results[0]
        html = result['html'][0]
        home_url = result['url'][0]
        domain = get_domain(home_url)
        
        # print ("Working on " + firm_name + ' at ' + home_url)
        soup = BeautifulSoup(html, 'lxml')
        links_to_text_dict = {a.get('href'): a.string for a in soup.find_all('a', href=True)}
        link_paths_to_text_dict = {get_link_path(link): text for link, text in links_to_text_dict.items() }
        pp.pprint(link_paths_to_text_dict)
        regex = re.compile('(about|company|corporate|who.we.are|(^|/)vision|overview|profile|corporate)', flags=re.IGNORECASE)
        about_links = {link: text for link, text in link_paths_to_text_dict.items() if regex.search(link)}
        about_texts = {link: text for link, text in link_paths_to_text_dict.items() if regex.search(text)}
        abouts = {**about_texts, **about_links}
        
        abouts_clnd = {link: text for link, text in abouts.items() if not(re.search('linkedin', link))}
        abouts_clnd = {fix_url(link, domain): text for link, text in abouts_clnd.items()} 
        about_keys_srtd = sorted(abouts_clnd.keys(), key=len )
        # pp.pprint (links)
        
        if abouts_srtd:
            firm_about_dict[firm_name] = abouts_srtd
        else: 
            firm_no_about_dict[firm_name] = home_url

get_about_links_from_home_pages()

{'': None,
 '/': u'Home',
 '/about-us': u'About Us',
 '/contact-us': u'Contact Us',
 '/features/downloads': u'How To',
 '/features/faqs': u'FAQs',
 '/features/gallery': u'PICS',
 '/features/stores': u'Stores',
 '/little_kids_inc': None,
 '/little_kids_inc/': None,
 '/littlekidsinc/': None,
 '/privacy': u'Privacy',
 '/products': u'Products',
 '/products/candylicious-bubbles': u'CANDYLICIOUS\xae',
 '/products/fubbles': u'Learn More',
 '/products/fubbles-no-spill': u'Fubbles\xae No-Spill\xae',
 '/products/jelly-belly': u'Jelly Belly\xae',
 '/products/junk-ball': u'Learn More',
 '/products/magic-kidchen': u'Magic Kidchen \u2122',
 '/products/nickelodeon': u'Nickelodeon \u2122',
 '/products/paw-patrol': u'Learn More',
 '/products/peeps': u'Learn More >>',
 '/products/pj-masks': u'PJ Masks \u2122',
 '/products/sesame-street': u'Sesame Street\xae',
 '/products/shimmer-shine': u'Shimmer & Shine \u2122',
 '/products/slick-tricks-bubbles': None,
 '/returns': u'Return Policy',
 '/terms': u'Terms'

NameError: global name 'link_paths' is not defined

In [134]:
# look to see which firms didn't get identified with having about pages
# may want to predict which page(s) in firm website have about us like information given the training sample of positive pages
print (len(firm_no_about_dict.keys()))
for firm_key, home_page in firm_no_about_dict.items():
    print (firm_key + ': ' + home_page)

141
Star Technology and Research: http://star-tech-inc.com/
NanoGram Corporation: http://www.nanogram.com/home/
Quallion LLC: https://www.enersys.com/GlobalLanding.aspx
Ivoclar Vivadent AG: http://www.ivoclarvivadent.com/
Forest Concepts: http://forestconcepts.com/
Mitsubishi Polyester Film: https://www.m-petfilm.com/
INENTEC INC.: http://inentec.com/
Industrial Technology Research Institute: https://www.itri.org.tw/eng/
PAX Scientific: http://paxscientific.com/
Biogenic Reagents Ventures: http://biogenicreagents.com/
Courtagen Life Sciences: http://courtagen.com/
KR Design House: http://krdesignhouse.com/
Ostendo Technologies: https://www.ostendo.com/
Masimo Semiconductor: http://masimosemiconductor.com/masimo-semiconductor/
Avertech: http://avertech.com.sg/
Graphene Technologies: http://graphenetechnologies.com/
Carver Scientific: http://carversci.com/
MEDport: https://medport.com/
Fina Technology: http://www.finatg.com/
Cleanvantage LLC: http://biovantage.dk/
Molecular Rebar Design:

In [146]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.10; rv:62.0) Gecko/20100101 Firefox/62.0'}
timeout = 5

def is_valid_url (url):
    try:
        resp = requests.get(url, headers=headers, timeout=timeout, verify=False)
        # print ("\tStatus code: " + str(resp.status_code))
        if resp.status_code >= 200 and resp.status_code < 400:
            return True
        else:
            return False
    except Exception as e: 
        print(e)
        return False
        # traceback.print_exc()

In [147]:
final_about_dict = {}
for firm_key, abouts in firm_about_dict.items():
    for url in abouts:
        if is_valid_url (url):
            final_about_dict[firm_key] = url
            break
    home = get_domain(abouts[0])
    firm_no_about_dict[firm_key] = home
    final_about_dict[firm_key] = home

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


HTTPSConnectionPool(host='www.magna.com', port=443): Read timed out. (read timeout=5)


/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


HTTPSConnectionPool(host='braunintertec.com', port=443): Max retries exceeded with url: //about-us (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x136fa6d10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
HTTPSConnectionPool(host='braunintertec.com', port=443): Max retries exceeded with url: //about-us/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x136fa6510>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
HTTPSConnectionPool(host='braunintertec.com', port=443): Max retries exceeded with url: //company/62021/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x136fa6990>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
HTTPSConnectionPool(host='braunintertec.com', port=443): Max retries exceeded with url: //about-us/awards/ (

...

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/sarora/anaconda2/lib/python2.7/site-packages/urllib3/connectionpool.py:858: In

In [ ]:
# check again since some firms may have been added 
print (len(firm_no_about_dict.keys()))

In [110]:
# write about pages to file
pp.pprint(final_about_dict)

{u'Little Kids': u'http://www.littlekidsinc.com'}
